In [1]:
import random
import weave
weave.use_frontend_devmode()
from rich import print
from weave.ecosystem import scenario

#weave.panels.Each.WeaveType().to_dict()
#scenario.metrics_bank.to_dict()

In [2]:
a = weave.save(5)
b = a + 2
weave.type_of(b)

Function(input_types={}, output_type=<number>)

In [3]:
def make_run(n_scenarios) -> list[scenario.ScenarioResult]:
    return [
        scenario.ScenarioResult(
            scenario_id=str(i),
            metric1=random.random(),
            metric2=random.random(),
            metric3=random.random())
        for i in range(n_scenarios)
    ]

In [4]:
panel_input = weave.save({'baseline': make_run(10), 'candidate': make_run(10)}, name='panel_input')

In [5]:
panel_input

In [8]:
import weave
weave.use_frontend_devmode()
from weave.ecosystem import scenario

# TODO
from weave import infer_types
scenario_weave_type = infer_types.python_type_to_type(scenario.ScenarioResult)

scenarios = weave.ops.objects(scenario_weave_type, 'latest')
panel = weave.panels.Group2(
    items={
        "baseline": weave.panels.ObjectPicker(scenarios),
        "candidate": weave.panels.ObjectPicker(scenarios),
        "panel": lambda baseline, candidate: scenario.metrics_bank(
            weave.ops.dict_(
                baseline=baseline.choice,
                candidate=candidate.choice))
    })
panel

INJECTED ObjectPicker(id='ObjectPicker', input_node=<OutputNode(4412495376): List(object_type=TypedDict(property_types={'scenario_id': <string>, 'metric2': <float>, 'metric1': <float>, 'metric3': <float>})) objects(TypedDict(property_types={'scenario_id': <string>, 'metric2': <float>, 'metric1': <float>, 'metric3': <float>}), "latest")>, vars={}, config=PanelObjectPickerConfig(choice=<VoidNode(5606182432): <invalid> <void>>))
CHILD ObjectPicker(id='ObjectPicker', input_node=<OutputNode(4412495376): List(object_type=TypedDict(property_types={'scenario_id': <string>, 'metric2': <float>, 'metric1': <float>, 'metric3': <float>})) objects(TypedDict(property_types={'scenario_id': <string>, 'metric2': <float>, 'metric1': <float>, 'metric3': <float>}), "latest")>, vars={}, config=PanelObjectPickerConfig(choice=<VoidNode(5606182432): <invalid> <void>>))
INJECTED ObjectPicker(id='ObjectPicker', input_node=<OutputNode(4412495376): List(object_type=TypedDict(property_types={'scenario_id': <string>

In [7]:
# What if I want to make this as a Python panel? I'd rather just make it in the UI.
weave.panels.ObjectPicker()